In [2]:
# Libraries
import numpy as np
import pandas as pd
import re

In [3]:
import sys
sys.path.append("..")
from src.cleaning import *

# Primera visualización rápida

Cargamos datos de formato CSV

In [4]:
attacks = pd.read_csv("../data/attacks.csv", encoding = "ISO-8859-1")

Visualizo las primeras y últimas filas para echar una ojeada a los datos.

In [5]:
attacks.head(2)

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,...,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN


In [6]:
attacks.tail(2)

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
25721,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25722,xx,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Visualizo también una muestra aleatoria.

In [7]:
attacks.sample(2)

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
12602,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22282,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Mis HIPOTESIS.

1_ USA es el pais donde más accidentes de tiburón se producen.

2_ La mayor parte de los accidentes son no provocados.

3_ Fishing y Surfing; mayores causas de accidentes.

4_ Mueren más hombres que mujeres.

# Análisis exploratorio del DataFrame

Evaluar la forma, "info" y "Nan" del Data Frame. Para este caso no merece la pena hacer un "describe". 

In [8]:
attacks.shape

(25723, 24)

In [9]:
attacks.info

<bound method DataFrame.info of       Case Number         Date    Year        Type    Country  \
0      2018.06.25  25-Jun-2018  2018.0     Boating        USA   
1      2018.06.18  18-Jun-2018  2018.0  Unprovoked        USA   
2      2018.06.09  09-Jun-2018  2018.0     Invalid        USA   
3      2018.06.08  08-Jun-2018  2018.0  Unprovoked  AUSTRALIA   
4      2018.06.04  04-Jun-2018  2018.0    Provoked     MEXICO   
...           ...          ...     ...         ...        ...   
25718         NaN          NaN     NaN         NaN        NaN   
25719         NaN          NaN     NaN         NaN        NaN   
25720         NaN          NaN     NaN         NaN        NaN   
25721         NaN          NaN     NaN         NaN        NaN   
25722          xx          NaN     NaN         NaN        NaN   

                  Area                        Location     Activity  \
0           California     Oceanside, San Diego County     Paddling   
1              Georgia  St. Simon Island, Gly

In [10]:
attacks.isnull().sum()

Case Number               17021
Date                      19421
Year                      19423
Type                      19425
Country                   19471
Area                      19876
Location                  19961
Activity                  19965
Name                      19631
Sex                       19986
Age                       22252
Injury                    19449
Fatal (Y/N)               19960
Time                      22775
Species                   22259
Investigator or Source    19438
pdf                       19421
href formula              19422
href                      19421
Case Number.1             19421
Case Number.2             19421
original order            19414
Unnamed: 22               25722
Unnamed: 23               25721
dtype: int64

Hago comprbación de DATOS NUMËRICOS y CATEGÓRICOS

In [11]:
# DATOS NUMÉRICOS
numericas = attacks._get_numeric_data()
numericas.head(3)

,Year,original order
0,2018.0,6303.0
1,2018.0,6302.0
2,2018.0,6301.0


In [13]:
# DATOS CATEGÓRICOS
attacks.select_dtypes(include="object").head(2)

,Case Number,Date,Type,Country,Area,Location,Activity,Name,Sex,Age,...,Time,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,Unnamed: 22,Unnamed: 23
0,2018.06.25,25-Jun-2018,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,...,18h00,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,NaN,NaN
1,2018.06.18,18-Jun-2018,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,11,...,14h00 -15h00,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,NaN,NaN


Renombro COLUMNAS. Elimino mayúsculas y espacios.

Nota; más adelante me di cuenta que tendría que renombrar de nuevo la columna "fata(y/n)" para poder trabajar sobre ella. Haré referncia a ello más adelante.

In [14]:
attacks_copia = attacks.copy()

In [15]:
# Saco los 5 primeros elementos.
columnas = list(attacks.columns)
columnas[:5]

['Case Number', 'Date', 'Year', 'Type', 'Country']

In [16]:
diccio_nuevas = {columna: columna.replace(" ", "_").lower() for columna in columnas}

In [17]:
diccio_nuevas

{'Case Number': 'case_number',
 'Date': 'date',
 'Year': 'year',
 'Type': 'type',
 'Country': 'country',
 'Area': 'area',
 'Location': 'location',
 'Activity': 'activity',
 'Name': 'name',
 'Sex ': 'sex_',
 'Age': 'age',
 'Injury': 'injury',
 'Fatal (Y/N)': 'fatal_(y/n)',
 'Time': 'time',
 'Species ': 'species_',
 'Investigator or Source': 'investigator_or_source',
 'pdf': 'pdf',
 'href formula': 'href_formula',
 'href': 'href',
 'Case Number.1': 'case_number.1',
 'Case Number.2': 'case_number.2',
 'original order': 'original_order',
 'Unnamed: 22': 'unnamed:_22',
 'Unnamed: 23': 'unnamed:_23'}

In [18]:
attacks_renombrado = attacks.rename(columns=diccio_nuevas)
attacks_renombrado.sample()

,case_number,date,year,type,country,area,location,activity,name,sex_,...,species_,investigator_or_source,pdf,href_formula,href,case_number.1,case_number.2,original_order,unnamed:_22,unnamed:_23
17561,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
attacks.rename(columns=diccio_nuevas, inplace=True)

In [20]:
attacks.head(2)

,case_number,date,year,type,country,area,location,activity,name,sex_,...,species_,investigator_or_source,pdf,href_formula,href,case_number.1,case_number.2,original_order,unnamed:_22,unnamed:_23
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,...,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN


Compruebo los datos que no son Nan en la columna unnamed:_23 para ver si hay algún dato relevante.

In [21]:

attacks[attacks["unnamed:_23"].notna()]

,case_number,date,year,type,country,area,location,activity,name,sex_,...,species_,investigator_or_source,pdf,href_formula,href,case_number.1,case_number.2,original_order,unnamed:_22,unnamed:_23
4415,1952.03.30,30-Mar-1952,1952.0,Unprovoked,NETHERLANDS ANTILLES,Curacao,NaN,Went to aid of child being menaced by the shark,A.J. Eggink,M,...,"Bull shark, 2.7 m [9'] was captured & dragged ...","J. Randall, p.352 in Sharks & Survival; H.D. B...",1952.03.30-Eggink.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1952.03.30,1952.03.30,1888.0,NaN,Teramo
5840,1878.09.14.R,Reported 14-Sep-1878,1878.0,Provoked,USA,Connecticut,"Branford, New Haven County",Fishing,Captain Pattison,M,...,NaN,"St. Joseph Herald, 9/14/1878",1878.09.14.R-Pattison.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1878.09.14.R,1878.09.14.R,463.0,NaN,change filename


Vuelvo a hacer un cambio de nombres en las columnas. Elimino los paréntesis de la columna "fatal" para poder utilizarla en la validación de hipótesis.

In [22]:
columnas_1 = list(attacks.columns)

In [23]:
diccio_nuevas_1 = {columna: columna.replace("(y/n)", "") for columna in columnas_1}

In [24]:
diccio_nuevas_1

{'case_number': 'case_number',
 'date': 'date',
 'year': 'year',
 'type': 'type',
 'country': 'country',
 'area': 'area',
 'location': 'location',
 'activity': 'activity',
 'name': 'name',
 'sex_': 'sex_',
 'age': 'age',
 'injury': 'injury',
 'fatal_(y/n)': 'fatal_',
 'time': 'time',
 'species_': 'species_',
 'investigator_or_source': 'investigator_or_source',
 'pdf': 'pdf',
 'href_formula': 'href_formula',
 'href': 'href',
 'case_number.1': 'case_number.1',
 'case_number.2': 'case_number.2',
 'original_order': 'original_order',
 'unnamed:_22': 'unnamed:_22',
 'unnamed:_23': 'unnamed:_23'}

In [25]:
attacks_renombrado = attacks.rename(columns=diccio_nuevas_1)
attacks_renombrado.sample()

,case_number,date,year,type,country,area,location,activity,name,sex_,...,species_,investigator_or_source,pdf,href_formula,href,case_number.1,case_number.2,original_order,unnamed:_22,unnamed:_23
23301,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
attacks.rename(columns=diccio_nuevas_1, inplace=True)

In [27]:
attacks.head(2)

,case_number,date,year,type,country,area,location,activity,name,sex_,...,species_,investigator_or_source,pdf,href_formula,href,case_number.1,case_number.2,original_order,unnamed:_22,unnamed:_23
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,...,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN


Ahora ELIMINO COLUMNAS y FILAS que no me aportan valor.

1- COLUMNAS

Elimino COLUMNAS que no aportan nada por su significado; no aportan nada a la tabla.

In [28]:
attacks.drop(["pdf", "href_formula","href","unnamed:_22", "unnamed:_23"],axis=1,inplace=True)

In [29]:
attacks.head(2)

,case_number,date,year,type,country,area,location,activity,name,sex_,age,injury,fatal_,time,species_,investigator_or_source,case_number.1,case_number.2,original_order
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,18h00,White shark,"R. Collier, GSAF",2018.06.25,2018.06.25,6303.0
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,11,Minor injury to left thigh,N,14h00 -15h00,NaN,"K.McMurray, TrackingSharks.com",2018.06.18,2018.06.18,6302.0


Aquí elimino columnas que no aportan nada para mis hipótesis.

In [30]:
attacks.drop(["time", "species_","investigator_or_source","case_number.1", "case_number.2", "original_order"],axis=1,inplace=True)

In [31]:
attacks.head(2)

,case_number,date,year,type,country,area,location,activity,name,sex_,age,injury,fatal_
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,11,Minor injury to left thigh,N


2-FILAS.

Elimino las filas en las que sus valores son todo NaN; no aporta nada a mi análisis. 

Para este paso he cambiado de data frame por si metía la pata. A apartir de aquí trabajo con el data frame "sharks"

In [32]:
sharks = attacks.dropna(how="all", axis=0)

In [33]:
sharks.shape

(8703, 13)

# LIMPIEZA por columnas.
COLUMNA; case_number

Elimino los ".letra" (.a,.b, .c, .d, .R) de todos los case_number. Aunque no aporta valor, como hice ya esta limpieza de esta columna, lo dejo indicado. Las fórmulas son repetitivas.

In [34]:
case_number = list(attacks.case_number.unique())

In [35]:
"""def quitar_a(x):
    if ".a" in str(x):
        return x.replace(".a", "")
    else:
        return x"""

'def quitar_a(x):\n    if ".a" in str(x):\n        return x.replace(".a", "")\n    else:\n        return x'

In [36]:
quitar_a

<function src.cleaning.quitar_a(x)>

In [37]:
sharks["case_number"] = sharks.case_number.apply(quitar_a)

<ipython-input-37-b9e4096ae751>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sharks["case_number"] = sharks.case_number.apply(quitar_a)


In [38]:
"""def quitar_b(x):
    if ".b" in str(x):
        return x.replace(".b", "")
    else:
        return x"""

'def quitar_b(x):\n    if ".b" in str(x):\n        return x.replace(".b", "")\n    else:\n        return x'

In [39]:
quitar_b

<function src.cleaning.quitar_b(x)>

In [40]:
sharks["case_number"] = sharks.case_number.apply(quitar_b)

<ipython-input-40-7a0d2429a33b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sharks["case_number"] = sharks.case_number.apply(quitar_b)


In [41]:
"""def quitar_c(x):
    if ".c" in str(x):
        return x.replace(".c", "")
    else:
        return x"""

'def quitar_c(x):\n    if ".c" in str(x):\n        return x.replace(".c", "")\n    else:\n        return x'

In [42]:
quitar_c

<function src.cleaning.quitar_c(x)>

In [43]:
sharks["case_number"] = sharks.case_number.apply(quitar_c)

<ipython-input-43-560dba157aa4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sharks["case_number"] = sharks.case_number.apply(quitar_c)


In [44]:
"""def quitar_d(x):
    if ".c" in str(x):
        return x.replace(".d", "")
    else:
        return x"""

'def quitar_d(x):\n    if ".c" in str(x):\n        return x.replace(".d", "")\n    else:\n        return x'

In [45]:
quitar_d

<function src.cleaning.quitar_d(x)>

In [46]:
sharks["case_number"] = sharks.case_number.apply(quitar_d)

<ipython-input-46-327631ea2d19>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sharks["case_number"] = sharks.case_number.apply(quitar_d)


In [47]:
"""def quitar_R(x):
    if ".R" in str(x):
        return x.replace(".R", "")
    else:
        return x"""

'def quitar_R(x):\n    if ".R" in str(x):\n        return x.replace(".R", "")\n    else:\n        return x'

In [48]:
quitar_R

<function src.cleaning.quitar_R(x)>

In [49]:
sharks["case_number"] = sharks.case_number.apply(quitar_R)

<ipython-input-49-46223fb203ba>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sharks["case_number"] = sharks.case_number.apply(quitar_R)


COLUMNA; date

Para la columna "date" me quedo solo con mes y año. El resto lo elimino. Al igual que antes no aporta valor pero lo dejo indicado.

In [50]:
sharks['newcol'] = sharks['date'].str.extract(r"(\w+-\d+)")

<ipython-input-50-65ad4d2e00fd>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sharks['newcol'] = sharks['date'].str.extract(r"(\w+-\d+)")


In [51]:
sharks.sample(2)

,case_number,date,year,type,country,area,location,activity,name,sex_,age,injury,fatal_,newcol
2815,1986.07.26,26-Jul-1986,1986.0,Boat,AUSTRALIA,Western Australia,3 km west of Rottnest Island,Fishing,"5.4 m boat, occupant: Ivan Anderton",NaN,NaN,"No injury to occupants. Shark bit motor, lifti...",N,Jul-1986
5425,1909.01.17,17-Jan-1909,1909.0,Invalid,NaN,NaN,Near the equator,Jumped overboard,Thomas Butler,M,36,FATAL,NaN,Jan-1909


In [52]:
dates = list(sharks.date)
print(dates[0:5])

['25-Jun-2018', '18-Jun-2018', '09-Jun-2018', '08-Jun-2018', '04-Jun-2018']


In [53]:
newcoll= list(sharks.newcol)
print(newcoll[0:5])

['Jun-2018', 'Jun-2018', 'Jun-2018', 'Jun-2018', 'Jun-2018']


In [54]:
sharks.date=newcoll

/home/jesus_chao/anaconda3/envs/ironhack/lib/python3.8/site-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [55]:
# Compruebo que me ha cambiado las fechas.
print(sharks.date[0:5])

0    Jun-2018
1    Jun-2018
2    Jun-2018
3    Jun-2018
4    Jun-2018
Name: date, dtype: object


In [56]:
sharks.sample()

,case_number,date,year,type,country,area,location,activity,name,sex_,age,injury,fatal_,newcol
855,2011.10.09,Oct-2011,2011.0,Unprovoked,AUSTRALIA,Western Australia,Cottesloe Beach,Swimming,Bryn Martin,M,64,FATAL,Y,Oct-2011


In [57]:
# Como he cambiado la date, puedo eliminar ya newcol
sharks.drop(["newcol"],axis=1, inplace=True)

/home/jesus_chao/anaconda3/envs/ironhack/lib/python3.8/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [58]:
sharks.sample()

,case_number,date,year,type,country,area,location,activity,name,sex_,age,injury,fatal_
3012,1981.09.03,Sep-1981,1981.0,Unprovoked,USA,Florida,"Patrick Air Force Base, Brevard County",Surfing,Richard Corrack,M,25,Lacerations to 2 fingers of left hand,N


COLUMNA; year

Aquí lo que hago es eliminar el ".0" de los años.

In [59]:
"""def year_(x):
    if ".0" in str(x):
        return str(x).replace(".0", "")
    else:
        return x"""

'def year_(x):\n    if ".0" in str(x):\n        return str(x).replace(".0", "")\n    else:\n        return x'

In [60]:
year_

<function src.cleaning.year_(x)>

In [61]:
sharks["year"] = sharks.year.apply(year_)

<ipython-input-61-34703d824513>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sharks["year"] = sharks.year.apply(year_)


In [62]:
print(sharks.year)

0        2018
1        2018
2        2018
3        2018
4        2018
         ... 
8698      NaN
8699      NaN
8700      NaN
8701      NaN
25722     NaN
Name: year, Length: 8703, dtype: object


In [63]:
# Este es el listado final de todas mis años.
sharks.year.unique()

array(['2018', '2017', nan, '2016', '2015', '2014', '2013', '2012',
       '2011', '2010', '2009', '2008', '2007', '2006', '2005', '2004',
       '2003', '2002', '2001', '2000', '1999', '1998', '1997', '1996',
       '1995', '1984', '1994', '1993', '1992', '1991', '1990', '1989',
       '1969', '1988', '1987', '1986', '1985', '1983', '1982', '1981',
       '1980', '1979', '1978', '1977', '1976', '1975', '1974', '1973',
       '1972', '1971', '1970', '1968', '1967', '1966', '1965', '1964',
       '1963', '1962', '1961', '1960', '1959', '1958', '1957', '1956',
       '1955', '1954', '1953', '1952', '1951', '1950', '1949', '1948',
       '1848', '1947', '1946', '1945', '1944', '1943', '1942', '1941',
       '1940', '1939', '1938', '1937', '1936', '1935', '1934', '1933',
       '1932', '1931', '1930', '1929', '1928', '1927', '1926', '1925',
       '1924', '1923', '1922', '1921', '1920', '1919', '1918', '1917',
       '1916', '1915', '1914', '1913', '1912', '1911', '1910', '1909',
       '1

COLUMNA type.

Cambios hechos en esta columna: Cambio nombre de "Unprovoked" por "Accidental".Además he metido "Boatomg" y "Boating" en una sola columna "Boat" 

In [64]:
sharks.type.unique()

array(['Boating', 'Unprovoked', 'Invalid', 'Provoked', 'Questionable',
       'Sea Disaster', nan, 'Boat', 'Boatomg'], dtype=object)

In [65]:
"""def type_(x):
    if "Unprovoked" in str(x):
        return x.replace("Unprovoked", "Accidental")
    else:
        return x"""

'def type_(x):\n    if "Unprovoked" in str(x):\n        return x.replace("Unprovoked", "Accidental")\n    else:\n        return x'

In [66]:
type_

<function src.cleaning.type_(x)>

In [67]:
sharks["type"] = sharks.type.apply(type_)

<ipython-input-67-e5a48b295325>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sharks["type"] = sharks.type.apply(type_)


In [68]:
multi_p = sharks[(sharks.type == "Boatomg")]

In [69]:
multi_p

,case_number,date,year,type,country,area,location,activity,name,sex_,age,injury,fatal_
1152,2009.03.01,Mar-2009,2009,Boatomg,NEW ZEALAND,North Island,Taranaki,Fishing,"boat, occupants: Boyd Rutherford & Hamish Roper",M,NaN,"No injury to occupants, shark bit propeller",N


In [70]:
"""def type_1(x):
    if "Boatomg" in str(x):
        return x.replace("Boatomg", "Boat")
    else:
        return x"""

'def type_1(x):\n    if "Boatomg" in str(x):\n        return x.replace("Boatomg", "Boat")\n    else:\n        return x'

In [71]:
type_1

<function src.cleaning.type_1(x)>

In [72]:
sharks["type"] = sharks.type.apply(type_1)

<ipython-input-72-240476cb040b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sharks["type"] = sharks.type.apply(type_1)


In [73]:
"""def type_2(x):
    if "Invalid" in str(x):
        return x.replace("Invalid", "Injured/Fatal")
    else:
        return x"""

'def type_2(x):\n    if "Invalid" in str(x):\n        return x.replace("Invalid", "Injured/Fatal")\n    else:\n        return x'

In [74]:
type_2

<function src.cleaning.type_2(x)>

In [75]:
sharks["type"] = sharks.type.apply(type_2)

<ipython-input-75-0df61136e9dd>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sharks["type"] = sharks.type.apply(type_2)


In [76]:
"""def type_3(x):
    if "Boating" in str(x):
        return x.replace("Boating", "Boat")
    else:
        return x"""

'def type_3(x):\n    if "Boating" in str(x):\n        return x.replace("Boating", "Boat")\n    else:\n        return x'

In [77]:
type_3

<function src.cleaning.type_3(x)>

In [78]:
sharks["type"] = sharks.type.apply(type_3)

<ipython-input-78-75c94eba09d1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sharks["type"] = sharks.type.apply(type_3)


In [79]:
sharks.type.unique()

array(['Boat', 'Accidental', 'Injured/Fatal', 'Provoked', 'Questionable',
       'Sea Disaster', nan], dtype=object)

COLUMNA; Pais, área y localización

Área y localización no me afecta a mis hipótesis; no los toco aunque se puedan mejorar

CAMBIOS: He convertido las letras minusculas en mayúsculas y eliminado "?". He metido también "ENGLAND" en "UNITED KINGDOM"

In [80]:
countries = list(sharks.country.unique())

In [81]:
len(countries)

213

In [82]:
print(countries[0:20])

['USA', 'AUSTRALIA', 'MEXICO', 'BRAZIL', 'ENGLAND', 'SOUTH AFRICA', 'THAILAND', 'COSTA RICA', 'MALDIVES', 'BAHAMAS', 'NEW CALEDONIA', 'ECUADOR', 'MALAYSIA', 'LIBYA', nan, 'CUBA', 'MAURITIUS', 'NEW ZEALAND', 'SPAIN', 'SAMOA']


Ponemos todos los países en mayúsculas.

In [83]:
"""def upper_c(x):
    return str(x).upper()"""

'def upper_c(x):\n    return str(x).upper()'

In [84]:
upper_c

<function src.cleaning.upper_c(x)>

In [85]:
sharks["country"] = sharks.country.apply(upper_c)

<ipython-input-85-51f75b38b595>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sharks["country"] = sharks.country.apply(upper_c)


Reemplazar ENGLAND por UNITED KINGDOM

In [86]:
"""def england_(x):
    if "ENGLAND" in str(x):
        return x.replace("ENGLAND", "UNITED KINGDOM")
    else:
        return x"""

'def england_(x):\n    if "ENGLAND" in str(x):\n        return x.replace("ENGLAND", "UNITED KINGDOM")\n    else:\n        return x'

In [87]:
england_

<function src.cleaning.england_(x)>

In [88]:
sharks["country"] = sharks.country.apply(england_)

<ipython-input-88-9a5641fac9da>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sharks["country"] = sharks.country.apply(england_)


ELIMINO signo de interrogación.

In [89]:
"""def question_(x):
    if "?" in str(x):
        return x.replace("?", "")
    else:
        return x"""

'def question_(x):\n    if "?" in str(x):\n        return x.replace("?", "")\n    else:\n        return x'

In [90]:
question_

<function src.cleaning.question_(x)>

In [91]:
sharks["country"] = sharks.country.apply(question_)

<ipython-input-91-1d540e1a7bd3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sharks["country"] = sharks.country.apply(question_)


In [97]:
# Este es el listado final de todos mis paises.
c_ =list(sharks.country.unique())

In [98]:
c_[0:5]

['USA', 'AUSTRALIA', 'MEXICO', 'BRAZIL', 'UNITED KINGDOM']

COLUMNA; Activity

Aunque mis hipótesis afectan a actividades, para validarlo he optado por otra opción que no sea limpieza. Ver la validación. Se han creado columnas FALSE y TRUE.

In [93]:
activities = list(sharks.activity.unique())

In [95]:
activities[0:5]

['Paddling', 'Standing', 'Surfing', 'Free diving', 'Kite surfing']

COLUMNA: sex_

Limpio la columna para quedarme sólo con F, M o None

In [99]:
gender = list(sharks.sex_.unique())

In [100]:
gender

['F', 'M', nan, 'M ', 'lli', 'N', '.']

In [101]:
sharks.head(2)

,case_number,date,year,type,country,area,location,activity,name,sex_,age,injury,fatal_
0,2018.06.25,Jun-2018,2018,Boat,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N
1,2018.06.18,Jun-2018,2018,Accidental,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,11,Minor injury to left thigh,N


In [102]:
"""def gender_(x):
    special_characters = 'FM'
    if str(x) in special_characters:
        return x
    else:
        return str(x).replace(str(x), "None")"""

'def gender_(x):\n    special_characters = \'FM\'\n    if str(x) in special_characters:\n        return x\n    else:\n        return str(x).replace(str(x), "None")'

In [103]:
gender_

<function src.cleaning.gender_(x)>

In [104]:
sharks["genero"] = sharks.sex_.apply(gender_)

<ipython-input-104-0220de8c0124>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sharks["genero"] = sharks.sex_.apply(gender_)


In [105]:
sharks.head()

,case_number,date,year,type,country,area,location,activity,name,sex_,age,injury,fatal_,genero
0,2018.06.25,Jun-2018,2018,Boat,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,F
1,2018.06.18,Jun-2018,2018,Accidental,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,11,Minor injury to left thigh,N,F
2,2018.06.09,Jun-2018,2018,Injured/Fatal,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,M
3,2018.06.08,Jun-2018,2018,Accidental,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,Minor injury to lower leg,N,M
4,2018.06.04,Jun-2018,2018,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,M


In [106]:
gender = list(sharks.genero.unique())

In [107]:
gender

['F', 'M', 'None']

In [108]:
sharks.drop(["sex_"],axis=1,inplace=True)

/home/jesus_chao/anaconda3/envs/ironhack/lib/python3.8/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [110]:
sharks.head(2)

,case_number,date,year,type,country,area,location,activity,name,age,injury,fatal_,genero
0,2018.06.25,Jun-2018,2018,Boat,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,57,"No injury to occupant, outrigger canoe and pad...",N,F
1,2018.06.18,Jun-2018,2018,Accidental,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,11,Minor injury to left thigh,N,F


COLUMNA: fatal_

Limpio para quedarme con N, Y o None

In [111]:
fatal = list(sharks.fatal_.unique())

In [112]:
fatal

['N', 'Y', nan, 'M', 'UNKNOWN', '2017', ' N', 'N ', 'y']

In [ ]:
"""def fatal_(x):
    special_characters = 'NYy'
    if str(x) in special_characters:
        return x.upper()
    else:
        return str(x).replace(str(x), "None")"""

In [113]:
fatal_

<function src.cleaning.fatal_(x)>

In [114]:
sharks["fatal"] = sharks.fatal_.apply(fatal_)

<ipython-input-114-988055907673>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sharks["fatal"] = sharks.fatal_.apply(fatal_)


In [115]:
sharks.head(2)

,case_number,date,year,type,country,area,location,activity,name,age,injury,fatal_,genero,fatal
0,2018.06.25,Jun-2018,2018,Boat,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,57,"No injury to occupant, outrigger canoe and pad...",N,F,N
1,2018.06.18,Jun-2018,2018,Accidental,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,11,Minor injury to left thigh,N,F,N


In [116]:
fatal_ = list(sharks.fatal.unique())

In [117]:
fatal_

['N', 'Y', 'None']

In [118]:
sharks.drop(["fatal_"],axis=1,inplace=True)

/home/jesus_chao/anaconda3/envs/ironhack/lib/python3.8/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [119]:
sharks.head(2)

,case_number,date,year,type,country,area,location,activity,name,age,injury,genero,fatal
0,2018.06.25,Jun-2018,2018,Boat,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,57,"No injury to occupant, outrigger canoe and pad...",F,N
1,2018.06.18,Jun-2018,2018,Accidental,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,11,Minor injury to left thigh,F,N


# Validación HIPOTESIS

1- USA es donde más accidentes de tiburón se producen.

Cuento los valores de la columna para validar hipótesis.

In [120]:
sharks.country.value_counts()

NAN                      2451
USA                      2229
AUSTRALIA                1338
SOUTH AFRICA              579
PAPUA NEW GUINEA          134
                         ... 
DIEGO GARCIA                1
ARGENTINA                   1
THE BALKANS                 1
NORTH ATLANTIC OCEAN        1
CEYLON (SRI LANKA)          1
Name: country, Length: 206, dtype: int64

SE CUMPLE.

2- La mayor parte de los accidentes son no provocados.

Cuento los valores de la columna para validar.

In [121]:
sharks.type.value_counts()

Accidental       4595
Provoked          574
Injured/Fatal     547
Boat              341
Sea Disaster      239
Questionable        2
Name: type, dtype: int64

SE CUMPLE.

3- Fishing y Surfing; mayores causas de accidentes.

 Creo dos columnas Tue or False para datos que terminan en "ishing" y "urfing". Los cuento y cuento los fishing y surfing.

In [122]:
sharks["fishing"] = sharks["activity"].str.contains("ishing")

<ipython-input-122-bfcf38036315>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sharks["fishing"] = sharks["activity"].str.contains("ishing")


In [123]:
sharks["surfing"] = sharks["activity"].str.contains("urfing")

<ipython-input-123-d92449e05581>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sharks["surfing"] = sharks["activity"].str.contains("urfing")


In [124]:
sharks.head(2)

,case_number,date,year,type,country,area,location,activity,name,age,injury,genero,fatal,fishing,surfing
0,2018.06.25,Jun-2018,2018,Boat,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,57,"No injury to occupant, outrigger canoe and pad...",F,N,False,False
1,2018.06.18,Jun-2018,2018,Accidental,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,11,Minor injury to left thigh,F,N,False,False


In [125]:
sharks.fishing.value_counts()

False    4586
True     1172
Name: fishing, dtype: int64

In [126]:
sharks.surfing.value_counts()

False    4630
True     1128
Name: surfing, dtype: int64

In [127]:
sharks.isnull().sum()

case_number       1
date           2946
year           2403
type           2405
country           0
area           2856
location       2941
activity       2945
name           2611
age            5232
injury         2429
genero            0
fatal             0
fishing        2945
surfing        2945
dtype: int64

2945 nulls

Calculo el porcentaje de personas con accidente causado por surfing y fishing. Si mayor de 50% se valida.

In [128]:
suma_fish_surf = 1128 + 1172

In [129]:
Porcentaje = suma_fish_surf/(4630+1128)

In [130]:
Porcentaje

0.399444251476207

NO SE CUMPLE.

4- Mueren más hombres que mujeres.

Cuento los valores de la columna para validar.

In [131]:
hombres_muertos = sharks[(sharks.genero == "M") & (sharks.fatal == "Y")]

In [132]:
hombres_muertos.genero.value_counts()

M    1173
Name: genero, dtype: int64

In [133]:
mujeres_muertas = sharks[(sharks.genero == "F") & (sharks.fatal == "Y")]

In [134]:
mujeres_muertas.genero.value_counts()

F    107
Name: genero, dtype: int64

SE CUMPLE.

# Conclusión

Las HIPOTESIS 1, 2, y 4 son verdaderas la 3 es falsa.

Esta es el aspecto final del csv

In [135]:
sharks.head(3)

,case_number,date,year,type,country,area,location,activity,name,age,injury,genero,fatal,fishing,surfing
0,2018.06.25,Jun-2018,2018,Boat,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,57,"No injury to occupant, outrigger canoe and pad...",F,N,False,False
1,2018.06.18,Jun-2018,2018,Accidental,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,11,Minor injury to left thigh,F,N,False,False
2,2018.06.09,Jun-2018,2018,Injured/Fatal,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,48,Injury to left lower leg from surfboard skeg,M,N,False,True


Exporto CSV limpio.

In [136]:
sharks.to_csv("sharks.csv", index = False)